In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


Test Summary: | Pass  Total
Rectifier     |    6      6


Test.DefaultTestSet("Rectifier", Any[], 6, false)

In [13]:
r = Rectifier("softplus")

Rectifier("softplus")

In [14]:
hessian(r, 1.0)

0.15403270679109896

In [18]:
ForwardDiff.derivative(z->ForwardDiff.derivative(y->r(y), z),x)

0.11090354888959125